# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import flatiron_stats as fs

%matplotlib inline 

In [63]:
df = pd.read_csv("homepage_actions.csv")

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [65]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [66]:
df['group'].value_counts() # looks to be roughly even experiemnt size vs control

control       4264
experiment    3924
Name: group, dtype: int64

In [67]:
df['action'].value_counts() # these two are not roughly the same size

view     6328
click    1860
Name: action, dtype: int64


Hints: * Start investigating the id column: * 
How many viewers also clicked? * Are there any anomalies with the data; 
did anyone click who didn't view? * Is there any overlap between the control 
and experiment groups? * If so, how do you plan to account for this in 
your experimental design?

In [68]:
clicked = df[df.action=='click']['id'].unique()
print(len(clicked))

viewed = df[df.action=='view']['id'].unique()
print(len(viewed))

view_no_click = (len(viewed) - len(clicked))
print(view_no_click)

total = len(clicked) + view_no_click
print(total)

1860
6328
4468
6328


From here we can see that 1,860 viewers clicked, 4,468 viewers did not click and our clicked population plus our did-not-click population are equal to our total amount of viewers so no further actions are needed to clean up this portion of the data. 

In [69]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [70]:
ids = df['id']

In [71]:
id_dict = ids.value_counts().to_dict()

In [72]:
clicked_ids = []
clicked_bool = []

In [73]:
for x, y in id_dict.items():
    if y == 2:
        clicked_ids.append(x)

In [74]:
for x in df['id']:
    if x in clicked_ids:
        clicked_bool.append(True)
    else:
        clicked_bool.append(False)

In [75]:
df['clicked'] = clicked_bool

In [76]:
df.head(10)

,timestamp,id,group,action,clicked
0,2016-09-24 17:42:27.839496,804196,experiment,view,False
1,2016-09-24 19:19:03.542569,434745,experiment,view,False
2,2016-09-24 19:36:00.944135,507599,experiment,view,False
3,2016-09-24 19:59:02.646620,671993,control,view,False
4,2016-09-24 20:26:14.466886,536734,experiment,view,False
5,2016-09-24 20:32:25.712659,681598,experiment,view,False
6,2016-09-24 20:39:03.248853,522116,experiment,view,False
7,2016-09-24 20:57:20.336757,349125,experiment,view,True
8,2016-09-24 20:58:01.948663,349125,experiment,click,True
9,2016-09-24 21:00:12.278374,560027,control,view,False


In [77]:
df.drop_duplicates(subset = 'id', keep = 'first', inplace = True)

In [78]:
df['id'].value_counts()

487387    1
638477    1
810521    1
243228    1
769565    1
         ..
753063    1
840503    1
666424    1
369465    1
196608    1
Name: id, Length: 6328, dtype: int64

In [79]:
df = df.drop(columns = 'action')

In [80]:
df

,timestamp,id,group,clicked
0,2016-09-24 17:42:27.839496,804196,experiment,False
1,2016-09-24 19:19:03.542569,434745,experiment,False
2,2016-09-24 19:36:00.944135,507599,experiment,False
3,2016-09-24 19:59:02.646620,671993,control,False
4,2016-09-24 20:26:14.466886,536734,experiment,False
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,False
8184,2017-01-18 09:42:12.844575,755912,experiment,False
8185,2017-01-18 10:01:09.026482,458115,experiment,False
8186,2017-01-18 10:08:51.588469,505451,control,False


In [81]:
df['clicked'].value_counts()

False    4468
True     1860
Name: clicked, dtype: int64

In [82]:
df['group'].value_counts()

control       3332
experiment    2996
Name: group, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

NULL hypothesis: That those in the experiment group had the same ratio of clicks to views as the control group. 

ALT hypothesis: That those in the experiment group had a different ratio of clicks to views as the control group.

ALPHA: 0.05

I will perform a chi^2 test because I'm comparing two categories of numeric data to each other, I have counts of the intersections between two groups of categorical data. I'll be using the chi^2 test for independence because I want to see if the relationship between control vs experiment is independent of the relationship between click vs view. Example of Chi^2 test for independence: between marriage status and education level. 

I need to take the IDs that viewed that did not also click and turn click into a binary value

In [83]:
contingency_table = pd.crosstab(index=df["clicked"], columns=df["group"])
contingency_table

group,control,experiment
clicked,,
False,2400,2068
True,932,928


In [84]:
result = stats.chi2_contingency(contingency_table)
chi, p_val, dof, exp = result
print(result)

# this shows the P value is below 0.05 we can reject the null hypothesis. 

(6.712921132285345, 0.009571680497042266, 1, array([[2352.61946903, 2115.38053097],
       [ 979.38053097,  880.61946903]]))


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [85]:
contingency_table

group,control,experiment
clicked,,
False,2400,2068
True,932,928


In [86]:
control_click = contingency_table['control'][1]
control_no_click = contingency_table['control'][0]
experiment_click = contingency_table['experiment'][1]
experiment_no_click = contingency_table['experiment'][0]

total_control = control_click + control_no_click
total_experiment = experiment_click + experiment_no_click

In [87]:
control_click_rate = (control_click / total_control) 

print (f'In the control the clicked_rate was {control_click_rate}.')

In the control the clicked_rate was 0.2797118847539016.


In [88]:
total_experiment


2996

In [89]:
experiement_expected_clicks = total_experiment * control_click_rate

print (f'If the experiemnt click rate was the same as the control\'s click \
rate, than I would expect the experiemnt to have generated \
{experiement_expected_clicks} clicks.')

If the experiemnt click rate was the same as the control's click rate, than I would expect the experiemnt to have generated 838.0168067226891 clicks.


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [90]:
n = total_experiment
_p = control_click_rate
var = n * _p * (1-_p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [91]:
z_score = (experiment_click - experiement_expected_clicks)/std
print(z_score)

3.6625360854823588


In [92]:
p_value = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_value)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **These scores are too different from each other, implying that the experiment should be rerun.**  

In [93]:
p_val, p_value

(0.009571680497042266, 0.00012486528006951198)

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.